## Wczytanie bibliotek

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels
import statsmodels.api as sm
import statsmodels.formula.api as smf
import datetime as dt
import scipy.signal as ss
import xgboost as xgb
from sklearn.tree import DecisionTreeRegressor
import group_lasso
from sklearn.metrics import mean_squared_error

## Wczytanie zbiorów

In [2]:
holidays_events = pd.read_csv("https://www.dropbox.com/s/bxyamlpevkiwwoq/holidays_events.csv?dl=1")
holidays_events["holiday_type"] = holidays_events["type"]
holidays_events.drop(["type"],axis=1,inplace=True)
oil = pd.read_csv("https://www.dropbox.com/s/l6ln0ztl4m0pw3a/oil.csv?dl=1",parse_dates=['date'],index_col='date')
oil2 = pd.read_csv("https://www.dropbox.com/s/l6ln0ztl4m0pw3a/oil.csv?dl=1")
sample_submission = pd.read_csv("https://www.dropbox.com/s/68jjl61x6u3klos/sample_submission.csv?dl=1")
stores = pd.read_csv("https://www.dropbox.com/s/lcxn6r9bs2exguq/stores.csv?dl=1")
test = pd.read_csv("https://www.dropbox.com/s/cvdo1gn7r5lu2uz/test.csv?dl=1",index_col='id')
train = pd.read_csv("https://www.dropbox.com/s/s8p2b5awnuqfk0d/train.csv?dl=1",index_col='id')
transactions = pd.read_csv("https://www.dropbox.com/s/92fij9bcwt0e0cj/transactions.csv?dl=1")

c:\Users\ndzad\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


## Funkcje pomocnicze

Funkcja przygotowująca dane objaśniające. Łączy zbiory i zmienia zmienne kategoryczne na zmienne numeryczna metodą One Hot Encoding, a także grupuje po dniu lub miesiącu.

In [55]:
def przygotowanie_danych(df: pd.DataFrame, type, type_item):
    """
    Funkcja łącząca zbiory i zmieniająca zmienne kategoryczne na zmienne numeryczne. Funkcja jest przewidziana
    df - zbiór danych treningowych
    type - dane sklepowe ('family') / dane rodzinne ('store_nbr')
    type_item - konretny element zmiennej type np. dla type=="family" type_item=="AUTOMOTIVE"
    """
    #Łaczymy z pozostałymi zbiorami

    #Zbiór stores
    df = df.merge(stores,how="left",left_on=['store_nbr'],right_on=['store_nbr'])

    #Zbiór transactions
    df = df.merge(transactions,how="left",left_on=['date','store_nbr'],right_on=['date','store_nbr'])

    #Zbiór holidays_events
    df = df.merge(holidays_events,how="left",left_on=['date'],right_on=['date'])

    #Dodanie oil
    df = df.merge(oil2,how="left",left_on=['date'],right_on=['date'])

    #Interpolacja braków danych z oil
    df.interpolate(method ='linear', limit_direction ='backward', inplace=True)

    #Wybieramy family ze zbioru traningowego
    df_fam = df.loc[(df[type]==type_item)]
    
    #Dodajemy zmienne na dzień tygodnia i na miesiąc
    df_fam['dayofweek'] = pd.DatetimeIndex(df_fam['date']).dayofweek + 1
    df_fam['month'] = pd.to_datetime(df_fam['date']).dt.month

    #Usunięcie zmiennych, które nie będą zmieniane
    df_fam.drop([type,"description","transferred"],axis=1,inplace=True)
    if type == "family":
        type_opposite = "store_nbr"
    elif type == "store_nbr":
        type_opposite = "family"

    #One Hot Encoding
    df_fam = pd.get_dummies(df_fam,columns=["locale",type_opposite,"city", "state", "type", "cluster","locale_name", "holiday_type"],prefix=["locale",type_opposite,"city", "state", "type", "cluster","locale_name", "holiday_type"])
    
    return(df_fam)

In [ ]:
def dane_drzewo(df: pd.DataFrame, type, type_item):
    df2 =df.copy()
    df2["family"], uniques=pd.factorize(df2["family"])
    df2["city"], uniques=pd.factorize(df2["city"])
    df2["state"], uniques=pd.factorize(df2["state"])
    df2["type"], uniques=pd.factorize(df2["type"])
    df2["locale"], uniques=pd.factorize(df2["locale"])
    df2["locale_name"], uniques=pd.factorize(df2["locale_name"])
    df2["holiday_type"], uniques=pd.factorize(df2["holiday_type"])
    df2.fillna(0,inplace=True)

Podział danych na część treningową i walidacyjną.

In [ ]:
def podzial(df: pd.DataFrame, date='2016-06-01'):
    """
    Funkcja dzieli zbiór df na cześć treningową i walidacyjną względem date.
    """
    train = df.loc[(df['date']<date)].drop(["date"],axis=1)
    test = df.loc[(df['date']>=date)].reset_index(drop=True).drop(["date"],axis=1)
    return(train,test)

## Funkcje modelujące

Model XGBoost

In [82]:
def model_xgb(df1: pd.DataFrame,df2: pd.DataFrame):
    """
    Funkcja tworzy model XGBoost na danych df1 i waliduje na danych df2.
    """
    #Zbiory danych treningowy
    X = df1.drop(['sales'],axis=1).copy()
    Y = df1['sales'].copy()

    #Zbiory danych walidacyjnych 
    X_val = df2.drop(['sales'],axis=1)
    Y_val = df2['sales'].copy()

    #Dopasowanie modelu
    model = xgb.XGBRegressor().fit(X,Y)
    Y_pred = model.predict(X_val)

    #RMSE - jakość dopasowania
    RMSE = mean_squared_error(Y_val,Y_pred,squared=False)
    print(RMSE)

    return([model, RMSE])

Model regresji liniowej

In [ ]:
def model_ols(df1: pd.DataFrame,df2: pd.DataFrame):
    """
    Funkcja tworzy model regresji liniowej metodą najmniejszych kwadratów na danych df1 i waliduje na danych df2.
    """
    #Zbiory danych treningowy
    X = df1.drop(['sales'],axis=1).copy()
    Y = df1['sales'].copy()

    #Zbiory danych walidacyjnych 
    X_val = df2.drop(['sales'],axis=1)
    Y_val = df2['sales'].copy()

    #Dopasowanie modelu
    model = sm.OLS(Y,X).fit()
    Y_pred = model.predict(X_val)

    #RMSE - jakość dopasowania
    RMSE = mean_squared_error(Y_val,Y_pred,squared=False)
    print(RMSE)

    return([model, RMSE])

Model drzewa regresyjnego

In [ ]:
def model_tree(df1: pd.DataFrame,df2: pd.DataFrame):
    """
    Funkcja tworzy model drzewa regresyjnego na danych df1 i waliduje na danych df2.
    """
    #Zbiory danych treningowy
    X = df1.drop(['sales'],axis=1).copy()
    Y = df1['sales'].copy()

    #Zbiory danych walidacyjnych 
    X_val = df2.drop(['sales'],axis=1)
    Y_val = df2['sales'].copy()

    #Dopasowanie modelu
    model = DecisionTreeRegressor().fit(X,Y)
    Y_pred = model.predict(X_val)

    #RMSE - jakość dopasowania
    RMSE = mean_squared_error(Y_val,Y_pred,squared=False)
    print(RMSE)

    return([model, RMSE])

In [ ]:
def model_group_lasso(df1,df2,type):
    """
    Funkcja tworzy model drzewa regresyjnego na danych df1 i waliduje na danych df2.
    """
    #Zbiory danych treningowy
    X = df1.drop(['sales'],axis=1).copy()
    Y = df1['sales'].copy()

    #Zbiory danych walidacyjnych 
    X_val = df2.drop(['sales'],axis=1)
    Y_val = df2['sales'].copy()

    #Dopasowanie modelu
    model = group_lasso.GroupLasso(groups=[-1,-1, 1,  1,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  3,  3,  3,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  5,  5,  5,  5,  5,  5,  5,  5,  5,
        5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  6,  6,  6,  6,
        6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  7,  7,  7,  7,  7,
        8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
        9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
        9,  9,  9,  9,  9,  9,  9, 10, 10, 10, 10, 10, 10,-1]).fit(X,Y)
    Y_pred = model.predict(X_val)

    #RMSE - jakość dopasowania
    RMSE = mean_squared_error(Y_val,Y_pred,squared=False)
    print(RMSE)

    return([model, RMSE])

## Modelowanie

In [89]:
(train2,test2) = podzial(przygotowanie_danych(train,'family','AUTOMOTIVE'),'2016-06-01')

C:\Users\ndzad\AppData\Local\Temp/ipykernel_22508/789074995.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fam['dayofweek'] = pd.DatetimeIndex(df_fam['date']).dayofweek + 1
C:\Users\ndzad\AppData\Local\Temp/ipykernel_22508/789074995.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fam['month'] = pd.to_datetime(df_fam['date']).dt.month


In [ ]:
(model_xgb, RMSE_xgb) = model_xgb(train2,test2)

In [67]:
(train3,test3) = podzial(przygotowanie_danych(train,'store_nbr',1),'2016-06-01')

C:\Users\ndzad\AppData\Local\Temp/ipykernel_22508/789074995.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fam['dayofweek'] = pd.DatetimeIndex(df_fam['date']).dayofweek + 1
C:\Users\ndzad\AppData\Local\Temp/ipykernel_22508/789074995.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fam['month'] = pd.to_datetime(df_fam['date']).dt.month
c:\Users\ndzad\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

In [72]:
(model_xgb, RMSE_xgb) = model_xgb(train3,test3)

211.96339640260342


Wynik dla pojedynczego sklepu wychodzi dużo gorszy od wyniku dla pojedynczej rodziny, więc dalej przeanalizujemy model XGBoost na wszystkich rodzinach.

In [77]:
families = np.array(train["family"].unique())
family_models = []
family_RMSEs = []

In [90]:
for i in families:
    (train2,test2) = podzial(przygotowanie_danych(train,'family',i),'2016-06-01')
    (model_xgb, RMSE_xgb) = model_xgb(train2,test2)
    family_models.append(model_xgb)
    family_RMSEs.append(RMSE_xgb)

C:\Users\ndzad\AppData\Local\Temp/ipykernel_22508/789074995.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fam['dayofweek'] = pd.DatetimeIndex(df_fam['date']).dayofweek + 1
C:\Users\ndzad\AppData\Local\Temp/ipykernel_22508/789074995.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fam['month'] = pd.to_datetime(df_fam['date']).dt.month
c:\Users\ndzad\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

TypeError: 'XGBRegressor' object is not callable

- Outliery (zrobione)
- Grupowanie po family,typie sklep (uśrednienie sprzedaży po dacie)
- Agregacja i predykcja na miesiąc 